In [ ]:
#Importing libraries
import selenium
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re
from bs4 import BeautifulSoup
import urllib
from selenium.common.exceptions import NoSuchElementException 
from selenium.common.exceptions import WebDriverException
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import csv

In [ ]:
#Setting up chrome driver
from selenium.webdriver.support.wait import WebDriverWait
options=webdriver.ChromeOptions()
chrome_path = "C:/Users/sangh/Documents/Anaconda_files/Healthcare/chromedriver_win32/chromedriver.exe"
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("test-type")
options.add_argument("--dns-prefetch-disable")
driver = webdriver.Chrome(chrome_path,options=options)


In [ ]:
#Creating different lists to extract info from each page
name=[]
hospital=[]
bio=[]

In [ ]:
#Defining fucntion for extracting Demographics
def demoInfo(url):
    
    #Getting all the individual doctor page links
    list_of_hrefs = []
    
    doctor_list = driver.find_elements_by_class_name("uSearch-cards")

    for block in doctor_list:
        elements = block.find_elements_by_tag_name("a")
        for el in elements:
            list_of_hrefs.append(el.get_attribute("href"))

    print (list_of_hrefs)
    
    #Removing all unwanted characters in the link list 
    try:
        regex = re.compile(r'^tel:')
        href=[x for x in list_of_hrefs if not regex.match(x)]
    except:
        list_of_hrefs=list(filter(None,list_of_hrefs))
        regex = re.compile(r'^tel:')
        href=[x for x in list_of_hrefs if not regex.match(x)]
    finally:
        print (len(href))
    
    
    for i in range(len(href)):
        time.sleep(8)
        driver.get(href[i])     
        
        try:
            x=driver.find_element_by_xpath("//h1[@data-qa-target='ProviderDisplayName']")
            name.append(x.text)
        except:
            name.append("Unknown")
        try:
            h=driver.find_element_by_xpath("//span[@data-qa-target='hospital-name']")
            hospital.append(h.text)
        except:
            hospital.append("Unknown")    
        try:
            y=driver.find_element_by_xpath("//p[@data-qa-target='premium-biography']")
            bio.append(y.text)
        except:
            bio.append("Unknown")
        
    print (len(name))
    print (len(hospital))
    print (len(bio))

In [ ]:
pageLinks=[]
#Setting initial URL
url=driver.get("https://www.healthgrades.com/usearch?what=Obstetrics%20%26%20Gynecology&where=IL&pageNum=1&sort.provider=bestmatch&state=IL")
pageLinks.append(url)
listhref=[]

#Loop to get all the pageLinks for Doctors
for k in range(51):
    try:
      time.sleep(2)
      element=driver.find_element_by_xpath("//*[@id='usearch-container']/div/main/div[1]/ul/li/nav/ul/li[10]/a[@class='_3ZWfj']")
      driver.execute_script("arguments[0].click();", element)
      pageLinks.append(driver.current_url)
     
    except ConnectionResetError:
      continue
    except:
      element1=driver.find_element_by_xpath("//*[@id='usearch-container']/div/main/div[1]/ul/li/nav/ul/li[9]/a[@class='_3ZWfj']")
      driver.execute_script("arguments[0].click();", element1)
      pageLinks.append(driver.current_url)
      continue
pageLinks.append(driver.current_url)


In [ ]:
len(pageLinks)

In [ ]:
pageLinks.remove(None)
pageLinks.append(driver.current_url)
seen = set()
result = []
for item in pageLinks:
    if item not in seen:
        seen.add(item)
        result.append(item)

In [ ]:
#Getting all the demographic info
for i in range(len(result)):
    url=driver.get(result[i])
    print (result[i])
    time.sleep(1)
    demoInfo(url)

In [ ]:
#Checking length of lists 
print (len(name))
print (len(hospital))
print (len(bio))

In [ ]:
#Output to csv file
reviewDict={'DoctorName':name,'Affiliated Hospital':hospital, 'Bio':bio}
dataframe=pd.DataFrame(data=reviewDict)
dataframe
dataframe.to_csv('HG_Demo2.csv', header=True, mode='a')